<a href="https://colab.research.google.com/github/ssinlao/CS4210-Phishing-Website-Project/blob/main/Phishing_Websites.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phishing Website Modeling
Data Info:
* Dataset consists of 1828 websites use to check for 18 features
* 859-legitimate websites collected from Yahoo directory
* 969-phishing websites collected from Phishtank Millersmiles archives

### Cleaning the Data
1. import necessary libraries
2. load the arff file
3. see what features are included
4. check for missing and null values

In [1]:
import pandas as pd
import numpy as np
from scipy.io import arff

In [2]:
data, meta = arff.loadarff('phishing-data.arff')
df = pd.DataFrame(data)

for column in df.select_dtypes(include=['object']).columns:
  df[column] = df[column].str.decode('utf-8')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11055 entries, 0 to 11054
Data columns (total 31 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   having_IP_Address           11055 non-null  object
 1   URL_Length                  11055 non-null  object
 2   Shortening_Service          11055 non-null  object
 3   having_At_Symbol            11055 non-null  object
 4   double_slash_redirecting    11055 non-null  object
 5   Prefix_Suffix               11055 non-null  object
 6   having_Sub_Domain           11055 non-null  object
 7   SSLfinal_State              11055 non-null  object
 8   Domain_registration_length  11055 non-null  object
 9   Favicon                     11055 non-null  object
 10  port                        11055 non-null  object
 11  HTTPS_token                 11055 non-null  object
 12  Request_URL                 11055 non-null  object
 13  URL_of_Anchor               11055 non-null  ob

In [3]:
df.head()
df.describe()

,having_IP_Address,URL_Length,Shortening_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registration_length,Favicon,...,popUpWindow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
count,11055,11055,11055,11055,11055,11055,11055,11055,11055,11055,...,11055,11055,11055,11055,11055,11055,11055,11055,11055,11055
unique,2,3,2,2,2,2,3,3,2,2,...,2,2,2,2,3,2,2,3,2,2
top,1,-1,1,1,1,-1,1,1,-1,1,...,1,1,1,1,1,-1,1,0,1,1
freq,7262,8960,9611,9400,9626,9590,4070,6331,7389,9002,...,8918,10043,5866,7612,5831,8201,9516,6156,9505,6157


Deleting rows with missing values:

In [4]:
df.dropna()

,having_IP_Address,URL_Length,Shortening_Service,having_At_Symbol,double_slash_redirecting,Prefix_Suffix,having_Sub_Domain,SSLfinal_State,Domain_registration_length,Favicon,...,popUpWindow,Iframe,age_of_domain,DNSRecord,web_traffic,Page_Rank,Google_Index,Links_pointing_to_page,Statistical_report,Result
0,-1,1,1,1,-1,-1,-1,-1,-1,1,...,1,1,-1,-1,-1,-1,1,1,-1,-1
1,1,1,1,1,1,-1,0,1,-1,1,...,1,1,-1,-1,0,-1,1,1,1,-1
2,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,-1,1,-1,1,0,-1,-1
3,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,-1,-1,1,-1,1,-1,1,-1
4,1,0,-1,1,1,-1,1,1,-1,1,...,-1,1,-1,-1,0,-1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11050,1,-1,1,-1,1,1,1,1,-1,-1,...,-1,-1,1,1,-1,-1,1,1,1,1
11051,-1,1,1,-1,-1,-1,1,-1,-1,-1,...,-1,1,1,1,1,1,1,-1,1,-1
11052,1,-1,1,1,1,-1,1,-1,-1,1,...,1,1,1,1,1,-1,1,0,1,-1
11053,-1,-1,1,1,1,-1,-1,-1,1,-1,...,-1,1,1,1,1,-1,1,1,1,-1


Since removing duplicates will remove the majority of the dataset and leave us with 5 possible combinations, we will keep dupes

## Feature Selection
Use entropy and information gain to find what features are the most significant for predicting the target class


**Set up features and target class**

In [25]:
X = df.drop(columns=['Result'])
y = df['Result']

**Decision Tree Classifier to Rank Features**
* using entropy and information gain

In [42]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import pandas as pd

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

In [45]:
importances = clf.feature_importances_

print("Feature Importances based on Information Gain:")
print(pd.Series(clf.feature_importances_, index=X.columns).sort_values(ascending=False))

Feature Importances based on Information Gain:
SSLfinal_State                0.509352
URL_of_Anchor                 0.163536
web_traffic                   0.046178
Links_in_tags                 0.034992
Prefix_Suffix                 0.029970
having_Sub_Domain             0.026683
Links_pointing_to_page        0.022604
SFH                           0.019130
Request_URL                   0.018047
Google_Index                  0.014273
age_of_domain                 0.014153
Page_Rank                     0.012450
having_IP_Address             0.011799
Domain_registration_length    0.010128
DNSRecord                     0.010055
URL_Length                    0.008237
popUpWindow                   0.007101
Submitting_to_email           0.005449
having_At_Symbol              0.005000
Shortening_Service            0.004779
on_mouseover                  0.004730
HTTPS_token                   0.004217
Iframe                        0.003589
Redirect                      0.003578
Statistical_repor

Train model using features that greater than a threshold and compare accuracy

In [52]:
threshold = 0.0002
selected_features = X.columns[importances > threshold]

X_train_selected = X_train[selected_features]
X_test_selected = X_test[selected_features]

In [53]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

y_pred_all_features = clf.predict(X_test)

accuracy_all_features = accuracy_score(y_test, y_pred_all_features)
print("Accuracy with all features:", accuracy_all_features)

clf_selected = DecisionTreeClassifier(criterion='entropy', random_state=42)
clf_selected.fit(X_train_selected, y_train)

y_pred_selected_features = clf_selected.predict(X_test_selected)

accuracy_selected_features = accuracy_score(y_test, y_pred_selected_features)
print("Accuracy with selected features:", accuracy_selected_features)

Accuracy with all features: 0.9614109134760326
Accuracy with selected features: 0.9617123907145011


The accuracy only improves if the RightClick feature is removed since it is the only one less than the threshold.